In [1]:
import numpy as np
import pandas as pd

In [13]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.optimizers import Adam, SGD,RMSprop, Adadelta, Nadam
from tensorflow.keras.models import Model
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [52]:
print(tensorflow.__version__)

2.6.0


In [3]:
img_width = 224
img_height = 224
channels=3
batch_size = 32
num_classes=56

In [4]:
seed_train_validation = 1 # Must be same for train_ds and val_ds
shuffle_value = True

In [7]:
train_ds = keras.utils.image_dataset_from_directory(
directory ='new plates/train',
image_size = (224, 224),
seed = seed_train_validation,
    color_mode='rgb',
    label_mode = 'categorical',
    batch_size=32,
shuffle = shuffle_value)

val_ds = keras.utils.image_dataset_from_directory(
directory ='new plates/test',
    image_size = (224, 224),
seed = seed_train_validation,
    label_mode = 'categorical',
    batch_size=32,
color_mode = 'rgb',
shuffle = shuffle_value)

Found 8161 files belonging to 56 classes.
Found 280 files belonging to 56 classes.


In [8]:
AUTOTUNE = tensorflow.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)  # Add your desired number of units
output = Dense(num_classes, activation='softmax')(x)  # Replace num_classes with the number of classes in your task

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 224, 224, 3)  7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_2[0][0]            
______________________________________________________________________________________________

In [17]:
model.fit(train_ds, epochs=10, batch_size=32, validation_data=val_ds)

Epoch 1/10
256/256 [==============================] - 58s 175ms/step - loss: 3.1614 - accuracy: 0.2405 - val_loss: 6.9929 - val_accuracy: 0.0321
Epoch 2/10
256/256 [==============================] - 40s 157ms/step - loss: 2.2657 - accuracy: 0.4529 - val_loss: 2.6579 - val_accuracy: 0.3143
Epoch 3/10
256/256 [==============================] - 40s 157ms/step - loss: 1.8412 - accuracy: 0.5476 - val_loss: 2.2039 - val_accuracy: 0.4750
Epoch 4/10
256/256 [==============================] - 40s 157ms/step - loss: 1.5794 - accuracy: 0.6107 - val_loss: 2.0683 - val_accuracy: 0.4929
Epoch 5/10
256/256 [==============================] - 40s 157ms/step - loss: 1.3614 - accuracy: 0.6640 - val_loss: 1.9675 - val_accuracy: 0.5250
Epoch 6/10
256/256 [==============================] - 40s 157ms/step - loss: 1.1960 - accuracy: 0.7042 - val_loss: 1.8918 - val_accuracy: 0.5571
Epoch 7/10
256/256 [==============================] - 40s 157ms/step - loss: 1.0564 - accuracy: 0.7343 - val_loss: 1.8653 - val_ac